In [1]:
import numpy as np
from src.disentangling_model import DisentanglingModel
from src.disentanglement_error import disentanglement_error
from sklearn.metrics import accuracy_score
from keras import Sequential
from keras.layers import Dense, Input
from keras.callbacks import EarlyStopping
from keras_uncertainty.models import StochasticClassifier
from keras_uncertainty.layers import StochasticDropout

def entropy(probs, axis=-1, eps=1e-6):
    return -np.sum(probs * np.log(probs + eps), axis=axis)


def predictive_entropy(probs, axis=-1) -> np.ndarray:
    probs = np.mean(probs, axis=0)
    return entropy(probs, axis)


def expected_entropy(probs, eps=1e-6) -> np.ndarray:
    return -np.mean((probs * np.log(probs + eps)).sum(axis=-1), axis=0)


def mutual_information(probs) -> np.ndarray:
    return predictive_entropy(probs) - expected_entropy(probs)


class WineITModel(DisentanglingModel):

    def __init__(self):
        super().__init__()

    def fit(self, X, y):
        model = Sequential()
        model.add(Input(13))
        model.add(Dense(32, activation='relu'))
        model.add(StochasticDropout(0.2))
        model.add(Dense(32, activation='relu'))
        model.add(StochasticDropout(0.2))
        model.add(Dense(32, activation='relu'))
        model.add(StochasticDropout(0.2))
        model.add(Dense(3, activation='softmax'))

        model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
        model.fit(X, y, epochs=50, batch_size=32, callbacks=EarlyStopping(monitor='loss', patience=2), verbose=0)
        self.model = StochasticClassifier(model)


    def predict_disentangling(self, X):
        stochastic_preds = self.model.predict_samples(X, num_samples=25, batch_size=32)

        classifications = stochastic_preds.mean(axis=0).argmax(axis=1)
        aleatorics = expected_entropy(stochastic_preds)
        epistemics = mutual_information(stochastic_preds)
        return classifications, aleatorics, epistemics


    def score(self, y_true, y_pred):
        return accuracy_score(y_true, y_pred)

    def is_regression(self):
        return False

Keras Uncertainty will use standalone Keras backend

In [2]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

X, y = load_wine(return_X_y=True)

le = LabelEncoder()
y = le.fit_transform(y)

sc = StandardScaler()
X = sc.fit_transform(X)


kw_config = {"n_runs": 2,
             "dataset_sizes": [0.1, 0.25, 0.5, 0.75, 1.0],
             }

print(disentanglement_error(X, y, WineITModel(), kw_config=kw_config))

0.5284331014528647
